### Preparation

In [27]:
### Setup and load data

import pandas as pd
import json

# Load the moderation log
log_df = pd.read_csv('../output/moderation_log.csv')

print(f"Total posts in log: {len(log_df)}")

Total posts in log: 150


In [28]:
### Parse labels

# Parse labels from JSON strings to Python lists
log_df['expected_labels_list'] = log_df['expected_labels'].apply(
    lambda x: json.loads(x) if pd.notna(x) and x != '' else []
)
log_df['predicted_labels_list'] = log_df['predicted_labels'].apply(
    lambda x: json.loads(x) if pd.notna(x) and x != '' else []
)

# Create string versions for grouping (sorted for consistency)
log_df['expected_labels_str'] = log_df['expected_labels_list'].apply(
    lambda x: json.dumps(sorted(x))
)
log_df['predicted_labels_str'] = log_df['predicted_labels_list'].apply(
    lambda x: json.dumps(sorted(x))
)

# Calculate correctness by comparing label lists
log_df['is_correct'] = log_df.apply(
    lambda row: sorted(row['expected_labels_list']) == sorted(row['predicted_labels_list']),
    axis=1
)

print("Labels parsed successfully")


Labels parsed successfully


### Dataset Analysis

In [29]:
### Dataset Breakdown 

# Count posts by source and label combination
breakdown = log_df.groupby(['source', 'expected_labels_str']).size().reset_index(name='count')

summary_data = []
for label_str in sorted(breakdown['expected_labels_str'].unique()):
    label_list = json.loads(label_str)
    label_display = json.dumps(label_list) if label_list else "[]"
    
    real = breakdown[(breakdown['expected_labels_str'] == label_str) & 
                     (breakdown['source'] == 'real')]['count'].sum()
    generated = breakdown[(breakdown['expected_labels_str'] == label_str) & 
                          (breakdown['source'] == 'generated')]['count'].sum()
    total = real + generated
    
    summary_data.append({
        'Labels': label_display,
        'Real Posts': real,
        'Generated Posts': generated,
        'Total Posts': total
    })

summary_df = pd.DataFrame(summary_data)
print("Dataset Breakdown:")
print(summary_df.to_string(index=False))
print(f"\nTotal: {summary_df['Total Posts'].sum()} posts ({summary_df['Real Posts'].sum()} real, {summary_df['Generated Posts'].sum()} generated)")

Dataset Breakdown:
                                Labels  Real Posts  Generated Posts  Total Posts
  ["drug-approved", "supported-claim"]           9               21           30
["drug-approved", "unsupported-claim"]           8               21           29
                     ["drug-approved"]          12               19           31
                   ["drug-unapproved"]          12               18           30
                                    []           9               21           30

Total: 150 posts (50 real, 100 generated)


### Accuracy Metrics

In [30]:
### Calculate the Overall Accuracy

total = len(log_df)
correct = log_df['is_correct'].sum()
accuracy = correct / total if total > 0 else 0

print(f"Overall Accuracy: {correct}/{total} = {accuracy:.3f} ({accuracy*100:.1f}%)")

Overall Accuracy: 137/150 = 0.913 (91.3%)


In [31]:
### Per-Label Combination Metrics

# Calculate metrics treating each label combination as a separate class
all_combinations = set(log_df['expected_labels_str'].unique()) | set(log_df['predicted_labels_str'].unique())

results = []
for combo_str in sorted(all_combinations):
    combo_list = json.loads(combo_str)
    combo_display = json.dumps(combo_list) if combo_list else "[]"
    
    tp = sum((pred_str == combo_str) and (exp_str == combo_str) 
             for pred_str, exp_str in zip(log_df['predicted_labels_str'], log_df['expected_labels_str']))
    fp = sum((pred_str == combo_str) and (exp_str != combo_str) 
             for pred_str, exp_str in zip(log_df['predicted_labels_str'], log_df['expected_labels_str']))
    fn = sum((pred_str != combo_str) and (exp_str == combo_str) 
             for pred_str, exp_str in zip(log_df['predicted_labels_str'], log_df['expected_labels_str']))
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    results.append({
        'Labels': combo_display,
        'Precision': f"{precision:.3f}",
        'Recall': f"{recall:.3f}",
        'F1': f"{f1:.3f}",
        'TP': tp,
        'FP': fp,
        'FN': fn
    })

metrics_df = pd.DataFrame(results)
print(metrics_df.to_string(index=False))

                                Labels Precision Recall    F1  TP  FP  FN
  ["drug-approved", "supported-claim"]     0.929  0.867 0.897  26   2   4
["drug-approved", "unsupported-claim"]     0.920  0.793 0.852  23   2   6
                     ["drug-approved"]     0.912  1.000 0.954  31   3   0
                   ["drug-unapproved"]     0.824  0.933 0.875  28   6   2
                                    []     1.000  0.967 0.983  29   0   1


### Performance Metrics

In [32]:
### Performance Metrics: 

print("Performance Metrics:")
print(f"Average time per post: {log_df['time_seconds'].mean():.2f}s")
print(f"Average memory usage: {log_df['memory_mb'].mean():.2f} MB")
print(f"Average LLM calls: {log_df['llm_calls'].mean():.1f}")
print(f"Average FDA calls: {log_df['fda_calls'].mean():.1f}")
print(f"\nTotals:")
print(f"Total LLM calls: {log_df['llm_calls'].sum()}")
print(f"Total FDA calls: {log_df['fda_calls'].sum()}")

Performance Metrics:
Average time per post: 9.24s
Average memory usage: 0.38 MB
Average LLM calls: 2.0
Average FDA calls: 1.4

Totals:
Total LLM calls: 300
Total FDA calls: 214


### Error Analysis

Error dataset created with 13 errors
Saved to: aanlysis/output/error_analysis.csv

Error breakdown:
error_type
Wrong Labels      10
False Negative     2
False Positive     1
Name: count, dtype: int64


In [38]:
# Performance for posts with vs without claims
print("\n" + "=" * 80)
print("PERFORMANCE BY CLAIM STATUS")
print("=" * 80)

has_claim_df = log_df[log_df['has_claim'] == True]
no_claim_df = log_df[log_df['has_claim'] == False]

print(f"\nPosts WITH claims ({len(has_claim_df)} posts):")
print(f"  Average time: {has_claim_df['time_seconds'].mean():.2f}s")
print(f"  Average LLM calls: {has_claim_df['llm_calls'].mean():.2f}")
print(f"  Average FDA calls: {has_claim_df['fda_calls'].mean():.2f}")

print(f"\nPosts WITHOUT claims ({len(no_claim_df)} posts):")
print(f"  Average time: {no_claim_df['time_seconds'].mean():.2f}s")
print(f"  Average LLM calls: {no_claim_df['llm_calls'].mean():.2f}")
print(f"  Average FDA calls: {no_claim_df['fda_calls'].mean():.2f}")


PERFORMANCE BY CLAIM STATUS

Posts WITH claims (53 posts):
  Average time: 14.86s
  Average LLM calls: 3.17
  Average FDA calls: 2.26

Posts WITHOUT claims (97 posts):
  Average time: 6.17s
  Average LLM calls: 1.36
  Average FDA calls: 0.97


In [39]:
# Time distribution
print("\n" + "=" * 80)
print("TIME DISTRIBUTION")
print("=" * 80)

# Categorize posts by processing time
log_df['time_category'] = pd.cut(log_df['time_seconds'], 
                                  bins=[0, 5, 10, 15, 20, float('inf')],
                                  labels=['<5s', '5-10s', '10-15s', '15-20s', '>20s'])

time_dist = log_df['time_category'].value_counts().sort_index()
print("\nPosts by processing time:")
for category, count in time_dist.items():
    pct = (count / len(log_df)) * 100
    print(f"  {category}: {count} posts ({pct:.1f}%)")


TIME DISTRIBUTION

Posts by processing time:
  <5s: 34 posts (22.7%)
  5-10s: 58 posts (38.7%)
  10-15s: 36 posts (24.0%)
  15-20s: 15 posts (10.0%)
  >20s: 7 posts (4.7%)


### Error Analysis

In [41]:
### Error Analysis

# Create error dataset
incorrect = log_df[log_df['is_correct'] == False].copy()

# Add error type classification
error_types = []
for idx, row in incorrect.iterrows():
    exp_set = set(row['expected_labels_list'])
    pred_set = set(row['predicted_labels_list'])
    
    fp_labels = pred_set - exp_set
    fn_labels = exp_set - pred_set
    
    if len(fp_labels) > 0 and len(fn_labels) == 0:
        error_types.append("False Positive")
    elif len(fn_labels) > 0 and len(fp_labels) == 0:
        error_types.append("False Negative")
    else:
        error_types.append("Wrong Labels")

incorrect['error_type'] = error_types

# Select relevant columns for error analysis
error_dataset = incorrect[[
    'source', 'input', 'expected_labels', 'predicted_labels', 
    'error_type', 'drugs_detected', 'has_claim', 'claim_text',
    'time_seconds', 'llm_calls', 'fda_calls'
]].copy()

# Save to CSV in output directory
error_dataset.to_csv('outputs/error_analysis.csv', index=False)

print(f"Error dataset created with {len(error_dataset)} errors")
print(f"Saved to: analysis/outputs/error_analysis.csv")
print("\nError breakdown:")
print(error_dataset['error_type'].value_counts())

Error dataset created with 13 errors
Saved to: analysis/outputs/error_analysis.csv

Error breakdown:
error_type
Wrong Labels      10
False Negative     2
False Positive     1
Name: count, dtype: int64
